<a href="https://colab.research.google.com/github/Icrz11/CS114.K21/blob/master/Sacrasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"khangnguyenhuu","key":"48f6620ba815aa2812de52477b147c80"}'}

In [0]:
!pip install kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 109MB/s]


In [0]:
#extract sarcasm zip file
from zipfile import ZipFile
zf = ZipFile("/content/news-headlines-dataset-for-sarcasm-detection.zip")
zf.extractall()

In [0]:
#read data
import pandas as pd
df = pd.read_json("/content/Sarcasm_Headlines_Dataset.json", lines = True)
df.head()
df = df.drop(['article_link'], axis = 1)
print (df)

                                                headline  is_sarcastic
0      former versace store clerk sues over secret 'b...             0
1      the 'roseanne' revival catches up to our thorn...             0
2      mom starting to fear son's web series closest ...             1
3      boehner just wants wife to listen, not come up...             1
4      j.k. rowling wishes snape happy birthday in th...             0
...                                                  ...           ...
26704               american politics in moral free-fall             0
26705                            america's best 20 hikes             0
26706                              reparations and obama             0
26707  israeli ban targeting boycott supporters raise...             0
26708                  gourmet gifts for the foodie 2014             0

[26709 rows x 2 columns]


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26709 entries, 0 to 26708
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   headline      26709 non-null  object
 1   is_sarcastic  26709 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 417.5+ KB


In [0]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
import string 
from string import digits
from string import punctuation
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
for i in df['headline']:
  i = str(i)

    

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
#clean the data remove all digits and punctuation
hl_clean = []


for i in df['headline']:
  clean = i.translate(str.maketrans('', '', punctuation))
  clean = clean.translate(str.maketrans('', '', digits))
  hl_clean.append(clean)

print('Before: ')
print(df['headline'][37])
print ('After: ')
print (hl_clean[37])


Before: 
'moana' sails straight to the top of the box office with massive $81.1 million opening
After: 
moana sails straight to the top of the box office with massive  million opening


In [0]:
#tonization, break the headline to the lonely character like noun, pharse, ....
hl_tokens = []
for i in hl_clean:
  i = i.split()
  hl_tokens.append(i)

print ('before: ')
print (hl_clean[100])
print ('after: ')
print (hl_tokens[100])

before: 
demi lovato drops emotional nightingale music vid
after: 
['demi', 'lovato', 'drops', 'emotional', 'nightingale', 'music', 'vid']


In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Init Lemmatizer
lemmatizer = WordNetLemmatizer()

hl_lemmatized = []
for tokens in hl_tokens:
    lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
    hl_lemmatized.append(lemm)



In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features = 2000
max_token = len (hl_lemmatized)
tokenizer = Tokenizer (num_words= max_features)
tokenizer.fit_on_texts(hl_lemmatized)
sequences = tokenizer.texts_to_sequences(hl_lemmatized)
X = pad_sequences(sequences, maxlen = max_token)

Using TensorFlow backend.


In [0]:
Y = df['is_sarcastic'].values
Y = np.vstack(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3, random_state = 42)
print (X_train)

[[   0    0    0 ...  647    2   22]
 [   0    0    0 ...  309   75  571]
 [   0    0    0 ...  218    7  824]
 ...
 [   0    0    0 ...  105    9  363]
 [   0    0    0 ...    5    2 1127]
 [   0    0    0 ... 1663   22    3]]


In [0]:

n_neighbors = 15
clf = knn = neighbors.KNeighborsClassifier(n_neighbors= 15)
clf.fit (train_set, training_labels)

ValueError: ignored